In [1]:
!pip install -U awscli boto3 sagemaker rich watermark --quiet

In [20]:
%load_ext watermark
%load_ext rich

%watermark -p awscli,boto3,sagemaker

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
The rich extension is already loaded. To reload it, use:
  %reload_ext rich
awscli   : 1.27.99
boto3    : 1.26.99
sagemaker: 2.141.0



In [ ]:
%%bash

IMAGE_NAME="abalonexgb-nginx"

docker build -t $IMAGE_NAME .

In [ ]:
%%bash

docker image list

In [ ]:
%%bash

# Run this in the terminal
# docker run -v $(pwd)/models:/opt/ml/model -p 8080:8080 --rm abalonexgb-inf-nginx

# ls -lahFg models
docker ps -a --format '{{.ID}}'

#### Launch container locally

Open a terminal and run the following command to test the container locally

`docker run -v $(pwd)/models:/opt/ml/model -p 8080:8080 --rm localinf`

### Ping container to check health

In [ ]:
%%bash

curl http://localhost:8080/ping

### Send a raw data record from `test.csv`

In [ ]:
%%bash

# Note last column is target
# curl --data-raw 'M,0.59,0.44,0.15,0.8725,0.387,0.215,0.245,8.0' -H 'Content-Type: text/csv' -v http://localhost:8080/invocations
# curl --data-raw 'F,0.585,0.465,0.145,0.9855,0.4325,0.2145,0.2845,10.0' -H 'Content-Type: text/csv' -v http://localhost:8080/invocations

### Tag the local image and push it to a private ECR

In [ ]:
%%bash

# Let's tag localimage to something more relevant
algorithm_name="abalonexgb-nginx"

# Rename the docker tag with algorithm_name
# docker tag localinf abalonexgb-inf-nginx

# Untag the old image
# docker image rm localinf

account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
echo $fullname

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### Upload model artifacts as `model.tar.gz` to S3


In [34]:
import tarfile
import os

model_path = os.path.join("./models", "xgboost-model")
model_output_path = os.path.join("./models", "model.tar.gz")

if not os.path.exists(model_output_path):
    print(f"Compressing model to {model_output_path}")
    tar = tarfile.open(model_output_path, "w:gz")
    tar.add(model_path, arcname="xgboost-model")
    tar.close()
else:
    print(f"Model file exists: {model_output_path}")

Model file exists: ./models/model.tar.gz

In [35]:
import sagemaker
import boto3
from sagemaker.s3 import S3Uploader, s3_path_join
from sagemaker import session
from sagemaker import get_execution_role
from rich import print

sm_session = session.Session()
region = sm_session._region_name
role = get_execution_role()
bucket = sm_session.default_bucket()
sm_client = boto3.client('sagemaker')

prefix = "sagemaker/abalone"
account_id = boto3.client("sts").get_caller_identity().get("Account")
model_s3uri = s3_path_join(f"s3://{bucket}/{prefix}", "models/byoc-nginx")

print(f"Role: {role}")
print(f"Bucket: {bucket}")
print(f"Model base: {model_s3uri}")

S3Uploader.upload(local_path=model_output_path, desired_s3_uri=model_s3uri, sagemaker_session=sm_session)

Role: arn:aws:iam::726793866085:role/service-role/AmazonSageMaker-ExecutionRole-20220313T104021

Bucket: sagemaker-us-east-1-726793866085

Model base: s3://sagemaker-us-east-1-726793866085/sagemaker/abalone/models/byoc-nginx

's3://sagemaker-us-east-1-726793866085/sagemaker/abalone/models/byoc-nginx/model.tar.gz'

### Create model object with custom inference image

In [36]:
from datetime import datetime
from uuid import uuid4

image_name = "abalonexgb-nginx"
ecr_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"
suffix = f"{str(uuid4())[:5]}-{datetime.now().strftime('%d%b%Y')}"

model_data_url = s3_path_join(model_s3uri, "model.tar.gz")

print(f"model_image_uri: {ecr_image}")
model_name = f"AbaloneXGB-nginx-{suffix}"

print(f"Creating model : {model_name} with {model_data_url}")
model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": ecr_image, 
        "ModelDataUrl": model_data_url
    }
)

print(f"Response")
print(model_response['ModelArn'])

model_image_uri: 726793866085.dkr.ecr.us-east-1.amazonaws.com/abalonexgb-nginx:latest

Creating model : AbaloneXGB-nginx-1f1fd-24Mar2023 with 
s3://sagemaker-us-east-1-726793866085/sagemaker/abalone/models/byoc-nginx/model.tar.gz

Response

arn:aws:sagemaker:us-east-1:726793866085:model/abalonexgb-nginx-1f1fd-24mar2023

### Create Endpoint Config

In [37]:
endpoint_config_name = f"AbaloneXGB-epc-nginx-{suffix}"
print(f"Creating endpoint config : {endpoint_config_name}")

# create endpoint config with data capture config using sagemaker boto3 client
epc_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.2xlarge",
            "InitialVariantWeight": 1,
        }
    ],
)

print(epc_response['EndpointConfigArn'])

Creating endpoint config : AbaloneXGB-epc-nginx-1f1fd-24Mar2023

arn:aws:sagemaker:us-east-1:726793866085:endpoint-config/abalonexgb-epc-nginx-1f1fd-24mar2023

In [38]:
# create endpoint with endpoint config and endpoint_image_uri
endpoint_name = f"AbaloneXGB-ep-nginx-{suffix}"
print(f"Creating endpoint : {endpoint_name}")

ep_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

print(ep_response['EndpointArn'])

Creating endpoint : AbaloneXGB-ep-nginx-1f1fd-24Mar2023

arn:aws:sagemaker:us-east-1:726793866085:endpoint/abalonexgb-ep-nginx-1f1fd-24mar2023

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print(f"Endpoint Status: {status}")
# Get the waiter object
waiter = sm_client.get_waiter("endpoint_in_service")
# Apply the waiter on the endpoint
waiter.wait(EndpointName=endpoint_name)
# Get endpoint status using describe endpoint
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

Endpoint Status: Creating

### Send test inference

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from time import time, sleep

runtime_sm_client = boto3.client('sagemaker-runtime')

LOCALDIR="."

local_test_dataset = f"{LOCALDIR}/test.csv"

test_s3uri = s3_path_join(f"s3://{bucket}/{prefix}", "data/test/test.csv")

# Download test data from S3
if not os.path.exists(local_test_dataset):
    print(f"Downloading test data to {LOCALDIR} from {test_s3uri} ...")
    S3Downloader.download(test_s3uri, local_path=LOCALDIR, sagemaker_session=sm_session)
else:
    print(f"Test file: {local_test_dataset} exists!")

limit=5
i = 0

with open(local_test_dataset, "r") as _f:
    for row in _f:
        if i == 0:
            print(f"Headers")
            print(row)
            print("---" * 20)
            i += 1
        elif i <= limit:
            row = row.rstrip("\n")
            splits = row.split(",")
            # Remove the target column
            label = splits.pop(-1)
            input_cols= ",".join(s for s in splits)
            
            try:
                print(f"Invoking EP with record")
                print(input_cols)
                prediction = runtime_sm_client.invoke_endpoint(
                    EndpointName=endpoint_name,
                    ContentType="text/csv",
                    Body=input_cols,
                )
                print(prediction["Body"].read())
                response = prediction["Body"].read().decode('utf-8')
                print(f"True: {label} | Predicted: {response}")
                sleep(0.5)
            except Exception as e:
                print(f"Prediction error: {e}")
                pass

In [32]:
from datetime import timedelta

logs_client = boto3.client('logs')
end_time = datetime.utcnow()
start_time = end_time - timedelta(minutes=15)

log_group_name = f'/aws/sagemaker/Endpoints/{endpoint_name}'
log_streams = logs_client.describe_log_streams(logGroupName=log_group_name)
log_stream_name = log_streams['logStreams'][0]['logStreamName']

# Retrieve the logs
logs = logs_client.get_log_events(
    logGroupName=log_group_name,
    logStreamName=log_stream_name,
    startTime=int(start_time.timestamp() * 1000),
    endTime=int(end_time.timestamp() * 1000)
)

# Print the logs
for event in logs['events']:
    print(f"{datetime.fromtimestamp(event['timestamp'] // 1000)}: {event['message']}")


2023-03-24 22:21:41: Starting the inference server with 8 workers.

2023-03-24 22:21:41: [2023-03-24 22:21:41 +0000] [10] [INFO] Starting gunicorn 20.1.0

2023-03-24 22:21:41: [2023-03-24 22:21:41 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)

2023-03-24 22:21:41: [2023-03-24 22:21:41 +0000] [10] [INFO] Using worker: sync

2023-03-24 22:21:41: [2023-03-24 22:21:41 +0000] [12] [INFO] Booting worker with pid: 12

...

2023-03-24 22:26:31: using cls.model directly

2023-03-24 22:26:31: Predictions obj

2023-03-24 22:26:31: <class 'list'>

2023-03-24 22:26:35: 169.254.178.2 - - [24/Mar/2023:22:26:31 +0000] "POST /invocations HTTP/1.1" 200 0 "-" 
"AHC/2.0"

2023-03-24 22:26:39: 169.254.178.2 - - [24/Mar/2023:22:26:35 +0000] "GET /ping HTTP/1.1" 404 1 "-" "AHC/2.0"

### Cleanup

In [33]:
# Delete endpoint, endpoint_configuration and model
print(f"EP: {endpoint_name}\nEPC: {endpoint_config_name}\nModel: {model_name}")

try:
    print(f"Deleting endpoint: {endpoint_name}")
    sm_client.delete_endpoint(EndpointName=endpoint_name)
except Exception as e:
    print(f"Error deleting EP: {endpoint_name}\n{e}")
    pass

try:
    print(f"Deleting endpoint_config: {endpoint_config_name}")
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
except Exception as e:
    print(f"Error deleting EPC: {endpoint_config_name}\n{e}")
    pass

try:
    print(f"Deleting Model: {model_name}")
    sm_client.delete_model(ModelName=model_name)
except Exception as e:
    print(f"Error deleting Model: {model_name}\n{e}")
    pass

EP: AbaloneXGB-ep-nginx-2162b-24Mar2023
EPC: AbaloneXGB-epc-nginx-2162b-24Mar2023
Model: AbaloneXGB-nginx-2162b-24Mar2023

Deleting endpoint: AbaloneXGB-ep-nginx-2162b-24Mar2023

Deleting endpoint_config: AbaloneXGB-epc-nginx-2162b-24Mar2023

Deleting Model: AbaloneXGB-nginx-2162b-24Mar2023